In [21]:
# 보기 편하게 이름 변경
# mp4 / xml 파일들
# size = 1920/1080
# 17개의 관절
# 골반, 왼힙, 왼무릎, 왼발, 오른힙, 오른무릎, 오른발. 등뼈, 척추, 목덜이, 머리, 오른어깨, 오른팔꿈치, 오른손, 왼어깨, 왼팔꿈치, 왼손

# 이상 행동, 구매 행동
# 프레임 별 각 관절의 좌표 움직임 차이 값을 시계열로 봅기
# 이상 행동의 차이 값이 더 클꺼라는 예상
# 평균 값 이상이라면 이상 행동

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # GPU 사용 비활성화

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 메모리 증가 허용
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [28]:
import xml.etree.ElementTree as ET

# XML 파일을 불러와 파싱
file_path = 'C:/Users/user/Desktop/train/label/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.xml'
tree = ET.parse(file_path)
root = tree.getroot()

# 각 'box' 태그에서 정보 추출
frame_data = {}
for box in root.iter('box'):
    frame = int(box.get('frame'))
    xtl = float(box.get('xtl'))
    ytl = float(box.get('ytl'))
    xbr = float(box.get('xbr'))
    ybr = float(box.get('ybr'))

    # 프레임별로 데이터 저장
    if frame not in frame_data:
        frame_data[frame] = []
    frame_data[frame].append((xtl, ytl, xbr, ybr))

# 추출된 데이터의 일부를 예시로 출력
example_frames = list(frame_data.keys())[:5]  # 첫 5개 프레임의 데이터를 예시로 출력
example_data = {frame: frame_data[frame] for frame in example_frames}
example_data

{74: [(1247.22, 162.78, 1526.39, 497.5)],
 75: [(1247.22, 162.78, 1526.39, 497.5)],
 146: [(976.39, 78.47, 1220.83, 692.36)],
 147: [(976.39, 78.47, 1220.83, 692.36)]}

: 

# 행동 시작과 끝 바운딩 좌표

In [33]:
import xml.etree.ElementTree as ET
import os
# XML 파일 경로
file_path = 'C:/Users/user/Desktop/train/label/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.xml'

tree = ET.parse(file_path)
root = tree.getroot()

yolo_format_data = {}

# 각 'box' 태그에서 정보 추출
for box in root.iter('box'):
    frame = int(box.get('frame'))
    xtl = float(box.get('xtl'))
    ytl = float(box.get('ytl'))
    xbr = float(box.get('xbr'))
    ybr = float(box.get('ybr'))

    img_width, img_height = 1920, 1080  # 크기 조정 필요
    x_center = ((xtl + xbr) / 2) / img_width
    y_center = ((ytl + ybr) / 2) / img_height
    width = (xbr - xtl) / img_width
    height = (ybr - ytl) / img_height


    class_index = 0  # 실제 클래스 인덱스로 수정 필요

    # 프레임별로 데이터 저장
    if frame not in yolo_format_data:
        yolo_format_data[frame] = []
    yolo_format_data[frame].append(f"{class_index} {x_center} {y_center} {width} {height}")


labels_dir = 'C:/Users/user/Desktop/SeSaC/Final_Project/labels'

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)

# 프레임별 데이터를 파일에 저장
for frame in yolo_format_data:
    file_path = os.path.join(labels_dir, f'frame_{frame}.txt')
    with open(file_path, 'w') as file:
        for item in yolo_format_data[frame]:
            file.write("%s\n" % item)


# 동영상 프레임 단위로 끊기
### 3프레임 = 1초
### 0~179

In [ ]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os
# 동영상이 있는 폴더
video_path = 'C:/Users/user/Desktop/train/video/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.mp4' 
frames_dir = 'frames'

if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)     # 폴더 자동 생성

cap = cv2.VideoCapture(video_path)
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_path = f'{frames_dir}/frame_{frame_num}.jpg'

    
    if frame is not None:
        cv2.imwrite(frame_path, frame)
        frame_num += 1

cap.release()

In [ ]:
import os

frame_dir = 'frames'  # 프레임이 저장된 디렉토리
frame_paths = [os.path.join(frame_dir, f) for f in sorted(os.listdir(frame_dir)) if f.endswith('.jpg')]
train_list_path = 'train.txt'

with open(train_list_path, 'w') as file:
    for path in frame_paths:
        file.write(path + '\n')


In [4]:
import xml.etree.ElementTree as ET
file_path = 'C:/Users/user/Desktop/train/label/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.xml'

tree = ET.parse(file_path)
root = tree.getroot()
label_data = {}

for track in root.iter('track'):
    label = track.get('label')
    for box in track.iter('box'):
        frame = int(box.get('frame'))
        xtl = float(box.get('xtl'))
        ytl = float(box.get('ytl'))
        xbr = float(box.get('xbr'))
        ybr = float(box.get('ybr'))

        if label not in label_data:
            label_data[label] = {}
        if frame not in label_data[label]:
            label_data[label][frame] = []
        
        label_data[label][frame].append((xtl, ytl, xbr, ybr))

example_labels = list(label_data.keys())[:3]  
example_label_data = {label: {frame: label_data[label][frame] for frame in list(label_data[label].keys())[:3]} for label in example_labels}
example_label_data

{'broken_start': {74: [(1247.22, 162.78, 1526.39, 497.5)],
  75: [(1247.22, 162.78, 1526.39, 497.5)]},
 'broken_end': {146: [(976.39, 78.47, 1220.83, 692.36)],
  147: [(976.39, 78.47, 1220.83, 692.36)]}}

# points 값 실패

In [29]:
# points 값 추출
right_foot_data = {}

for track in root.iter('track'):
    if track.get('label') == 'Right foot':
        for point in track.iter('points'):
            frame = int(point.get('frame'))
            points = point.get('points')  # 포인트 좌표

            if frame not in right_foot_data:
                right_foot_data[frame] = []
            right_foot_data[frame].append(points)


example_right_foot_frames = list(right_foot_data.keys())
example_right_foot_data = {frame: right_foot_data[frame] for frame in example_right_foot_frames}
example_right_foot_data
def calculate_point_differences(points):
    differences = []
    for i in range(len(points) - 1):
        x1, y1 = map(int, points[i].split(','))
        x2, y2 = map(int, points[i+1].split(','))
        diff = abs(x2 - x1), abs(y2 - y1)
        differences.append(diff)
    return differences

right_foot_differences = {frame: calculate_point_differences(right_foot_data[frame]) for frame in right_foot_data}


example_right_foot_differences = {frame: right_foot_differences[frame] for frame in list(right_foot_differences.keys())[1:-1]}
example_right_foot_points = {frame: right_foot_data[frame] for frame in right_foot_data}
example_right_foot_differences
example_right_foot_points

{74: ['1401,495'],
 75: ['1401,495', '1400,498'],
 76: ['1400,498', '1405,493'],
 77: ['1405,493', '1402,494'],
 78: ['1402,494', '1401,496'],
 79: ['1401,496', '1415,495'],
 80: ['1415,495', '1426,492'],
 81: ['1426,492', '1434,496'],
 82: ['1434,496', '1417,497'],
 83: ['1417,497', '1402,491'],
 84: ['1402,491', '1395,572'],
 85: ['1395,572', '1371,562'],
 86: ['1371,562', '1288,563'],
 87: ['1288,563', '1224,576'],
 88: ['1224,576', '1221,578'],
 89: ['1221,578', '1219,581'],
 90: ['1219,581', '1213,574'],
 91: ['1213,574', '1127,595'],
 92: ['1127,595', '985,576'],
 93: ['985,576', '955,586'],
 94: ['955,586', '946,582'],
 95: ['946,582', '895,584'],
 96: ['895,584', '835,697'],
 97: ['835,697', '838,699'],
 98: ['838,699', '836,699'],
 99: ['836,699', '839,698'],
 100: ['839,698', '836,699'],
 101: ['836,699', '838,698'],
 102: ['838,698', '839,694'],
 103: ['839,694', '837,652'],
 104: ['837,652', '867,597'],
 105: ['867,597', '867,587'],
 106: ['867,587', '869,585'],
 107: ['869

In [20]:
import xml.etree.ElementTree as ET

file_path = 'C:/Users/user/Desktop/train/label/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.xml'
tree = ET.parse(file_path)
root = tree.getroot()

labels = set(track.get('label') for track in root.iter('track'))
for i in labels:
    print(i)

Right  hip
Left knee
Pelvis
Center head
Right foot
Neck base
Right elbow
Right knee
Spine chest
Right shoulder
Right hand
Left hand
Spine naval
Left elbow
broken_start
Left hip
Left shoulder
Left foot
broken_end


In [26]:
def calculate_all_differences(root, labels):
    all_differences = {}

    for label in labels:
        label_data = {}

        for track in root.iter('track'):
            if track.get('label') == label:
                for point in track.iter('points'):
                    frame = int(point.get('frame'))
                    points = point.get('points')  

                    if frame not in label_data:
                        label_data[frame] = []
                    label_data[frame].append(points)

        label_differences = {frame: calculate_point_differences(label_data[frame]) for frame in label_data}
        all_differences[label] = label_differences

    return all_differences

all_label_differences = calculate_all_differences(root, labels)


example_all_label_differences = {label: {frame: all_label_differences[label][frame] for frame in list(all_label_differences[label].keys())[1:-1]} for label in labels}
right_hip = example_all_label_differences['Right  hip']  
left_knee = example_all_label_differences['Left knee']
pelvis = example_all_label_differences['Pelvis']
head = example_all_label_differences['Center head']
right_foot = example_all_label_differences['Right foot']
neck = example_all_label_differences['Neck base']
right_ellow = example_all_label_differences['Right elbow']
right_knee = example_all_label_differences['Right knee']
spine_chest = example_all_label_differences['Spine chest']
right_shoulder = example_all_label_differences['Right shoulder']
right_hand = example_all_label_differences['Right hand']
left_hand = example_all_label_differences['Left hand']
spint_naval = example_all_label_differences['Spine naval']
lest_elbow = example_all_label_differences['Left elbow']
left_hip = example_all_label_differences['Left hip']
left_shoulder = example_all_label_differences['Left shoulder']
left_foot = example_all_label_differences['Left foot']

In [34]:
pip install opencv-python torch torchvision

Note: you may need to restart the kernel to use updated packages.


# 이상 행동 부분만 추출 ex)74~147

In [48]:
import cv2
import os

cap = cv2.VideoCapture('C:/Users/user/Desktop/train/video/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.mp4')
frames_dir = "C:/Users/user/Desktop/train/result"
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(f'{frames_dir}/output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 이상 행동 프레임에 대한 처리
    if 74 <= frame_num <= 147:
        results = model(frame)

        # 결과 처리
        for result in results:
            out.write(frame)


        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    frame_num += 1

cap.release()
out.release()
cv2.destroyAllWindows()

# 모든 파일에 적용할 함수 만들기 필수


0: 384x640 1 person, 72.8ms
Speed: 0.0ms preprocess, 72.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 64.3ms
Speed: 0.0ms preprocess, 64.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 48.7ms
Speed: 1.6ms preprocess, 48.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 50.4ms
Speed: 0.0ms preprocess, 50.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 68.1ms
Speed: 0.0ms preprocess, 68.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 53.7ms
Speed: 2.0ms preprocess, 53.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 45.6ms
Speed: 1.0ms preprocess, 45.6ms inference, 11.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 59.9ms
Speed: 2.0ms preprocess, 59.9ms inference, 2.0ms postprocess per image at shape (1, 3, 3

In [33]:
import glob
import os
import xml.etree.ElementTree as ET
import cv2

xml_directory = 'C:/Users/user/Desktop/데이터/Labels/VL_03.이상행동_12.절도'

def process_video(file_path, start_frame, end_frame):
    cap = cv2.VideoCapture(file_path)
    frames_dir = "C:/Users/user/Desktop/데이터/abnormal"
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{frames_dir}/{os.path.basename(file_path)}_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if start_frame <= frame_num <= end_frame:
            out.write(frame)
            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        frame_num += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

for xml_file in glob.glob(os.path.join(xml_directory, '*.xml')):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for box in root.findall('.//box'):
        file_name = box.get('filename')
        start_frame = int(box.get('frame'))
        end_frame = int(box.get('frame'))

        video_path = f'C:/Users/user/Desktop/데이터/Video/VS_03.이상행동_12.절도/{file_name}.mp4'
        process_video(video_path, start_frame, end_frame)


In [36]:
import glob
# Revised code to process video based on 'theft_start' and 'theft_end' labels in the XML file

def extract_theft_frames(xml_file_path):
    """
    Extracts the start and end frames of theft events from the given XML file.
    """
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    theft_events = []

    for track in root.findall('.//track'):
        if track.get('label') in ['theft_start', 'theft_end']:
            for box in track.findall('.//box'):
                frame = int(box.get('frame'))
                if track.get('label') == 'theft_start':
                    theft_events.append({'start_frame': frame})
                elif track.get('label') == 'theft_end':
                    if theft_events and 'end_frame' not in theft_events[-1]:
                        theft_events[-1]['end_frame'] = frame

    return theft_events
xml_file_path = "C:/Users/user/Desktop/데이터/Labels/VL_03.이상행동_12.절도/*"
theft_frames = extract_theft_frames(xml_file_path)
theft_frames


OSError: [Errno 22] Invalid argument: 'C:/Users/user/Desktop/데이터/Labels/VL_03.이상행동_12.절도/*'

In [ ]:
import xml.etree.ElementTree as ET

file_path = 'C:/Users/user/Desktop/train/label/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.xml'
tree = ET.parse(file_path)
root = tree.getroot()

# 이상 행동 시작 및 종료 프레임 추출
start_frame = None
end_frame = None
for box in root.iter('box'):
    frame = int(box.get('frame'))
    # 여기에 이상 행동을 판단하는 로직을 추가
    # 예: if box.get('some_attribute') == 'some_value':
    # 이상 행동의 시작과 끝 프레임 결정
    if start_frame is None:
        start_frame = frame
    end_frame = frame

# 시작 프레임과 종료 프레임 출력
print("Start frame:", start_frame, "End frame:", end_frame)


In [ ]:
import os
import xml.etree.ElementTree as ET
def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    tracks = []
    for track_elem in root.findall('track'):
        track_id = int(track_elem.attrib['id'])
        label = track_elem.attrib['label']
        source = track_elem.attrib['source']
        boxes = []
        for box_elem in track_elem.findall('box'):
            frame = int(box_elem.attrib['frame'])
            xtl = float(box_elem.attrib['xtl'])
            ytl = float(box_elem.attrib['ytl'])
            xbr = float(box_elem.attrib['xbr'])
            ybr = float(box_elem.attrib['ybr'])
            attributes = {attr.attrib['name']: attr.text for attr in box_elem.findall('attribute')}
            box_info = {
                'frame': frame,
                'xtl': xtl,
                'ytl': ytl,
                'xbr': xbr,
                'ybr': ybr,
                'attributes': attributes
            }
            boxes.append(box_info)
        track_info = {
            'track_id': track_id,
            'label': label,
            'source': source,
            'boxes': boxes
        }
        tracks.append(track_info)
    return tracks
def extract_first_last_frames(xml_path):
    parsed_data = parse_xml(xml_path)
    broken_start_frames = []
    broken_end_frames = []
    for track_info in parsed_data:
        label = track_info['label'].lower()
        if 'broken_start' in label:
            broken_start_frames.append(track_info['boxes'][0]['frame'])
        elif 'broken_end' in label:
            broken_end_frames.append(track_info['boxes'][-1]['frame'])
    return broken_start_frames, broken_end_frames
def process_multiple_xml_files(xml_directory):
    xml_files = [file for file in os.listdir(xml_directory) if file.endswith('.xml')]
    all_first_last_frames = {}
    for xml_file in xml_files:
        xml_path = os.path.join(xml_directory, xml_file)
        first_frames, last_frames = extract_first_last_frames(xml_path)
        all_first_last_frames[xml_file] = { first_frames, last_frames}
    return all_first_last_frames
# 예시: XML 디렉토리 경로
xml_directory = 'C:/Users/user/Downloads/strange/train/labeling/damage/'
all_first_last_frames = process_multiple_xml_files(xml_directory)

In [30]:
import xml.etree.ElementTree as ET

file_path = 'C:/Users/user/Desktop/데이터/Labels/VL_03.이상행동_12.절도/C_3_12_40_BU_SMC_10-14_11-43-44_CD_RGB_DF2_M2.xml'

tree = ET.parse(file_path)
root = tree.getroot()

start_end = []
for box in root.iter('box'):
    frame = int(box.get('frame'))
    if box.get('label') == 'theft_start':  
        frame = int(box.get('frame'))
        start_end.append(frame)
    print(frame)

79
80
81
82
83
96
97
98
99
100
101
120
121
122
123
124
125
126
136
137
138
139
140
141
142
143
144
79
80
150
151


# MediaPipe 사용 키포인트 추출

### 데이터 전처리
- 프레임 분리 : 동영상을 개별 프레임으로 분리 각 프레임에서 키포인트 데이터를 추출
- 데이터 정제 : 누락 데이터, 이상치 제거
- 데이터 정규화
### 특징 추출
- 시계열 데이터 분석 : 시간에 따른 키포인트의 움직임을 분석
- 행동 특징 추출 : 속도, 가속도, 관절 각도 등 행동을 나타낼 수 있는 특징을 추출
- 패턴 인식 : 반복되는 움직임 또는 특정 패턴을 인식
### 이상 행동 정의

In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import json
import os
import glob

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

video_folder = 'C:/Users/user/Desktop/video/broken'
result_folder = 'C:/Users/user/Desktop/video/result'

if not os.path.exists(result_folder):
    os.makedirs(result_folder)
    
for video_file in glob.glob(os.path.join(video_folder, '*.mp4')):
    cap = cv2.VideoCapture(video_file)
    pose_landmarks_data = []
# 키포인트 데이터를 저장할 리스트
pose_landmarks_data = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # BGR 이미지를 RGB로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # 키포인트 추출 및 표시
    if results.pose_landmarks:
        # 키포인트 정보를 딕셔너리로 변환하여 저장
        landmarks = {}
        for i, landmark in enumerate(results.pose_landmarks.landmark):
            landmarks[f'x{i}'] = landmark.x
            landmarks[f'y{i}'] = landmark.y
            landmarks[f'z{i}'] = landmark.z
            landmarks[f'vis{i}'] = landmark.visibility
        pose_landmarks_data.append(landmarks)

        # 키포인트를 프레임에 그리기
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('MediaPipe Pose', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키를 누르면 종료
        break

cap.release()

# JSON 파일로 저장
with open('C:/Users/user/Desktop/video/result/pose_landmarks.json', 'w') as f:
    json.dump(pose_landmarks_data, f)

with open('C:/Users/user/Desktop/train/result/pose_landmarks.json', 'r') as f:
    data = json.load(f)

print(data)

[{'x0': 0.07935936748981476, 'y0': 0.1465035080909729, 'z0': -0.1190868467092514, 'vis0': 0.9689133763313293, 'x1': 0.06349990516901016, 'y1': 0.1303352415561676, 'z1': -0.11600973457098007, 'vis1': 0.9444217681884766, 'x2': 0.06267279386520386, 'y2': 0.13125640153884888, 'z2': -0.11622026562690735, 'vis2': 0.9404640197753906, 'x3': 0.06190153583884239, 'y3': 0.13224920630455017, 'z3': -0.11639464646577835, 'vis3': 0.9526078104972839, 'x4': 0.06368862092494965, 'y4': 0.13224956393241882, 'z4': -0.14455990493297577, 'vis4': 0.9432647228240967, 'x5': 0.06131189689040184, 'y5': 0.13526147603988647, 'z5': -0.14459143579006195, 'vis5': 0.9457177519798279, 'x6': 0.058969076722860336, 'y6': 0.1381751000881195, 'z6': -0.1447608917951584, 'vis6': 0.9551180601119995, 'x7': 0.04812737554311752, 'y7': 0.1639009416103363, 'z7': -0.09243936836719513, 'vis7': 0.9721264839172363, 'x8': 0.04690789431333542, 'y8': 0.1732993721961975, 'z8': -0.22083960473537445, 'vis8': 0.9778380990028381, 'x9': 0.081850

# 포즈 랜드 마크 추출

In [1]:
import cv2
import mediapipe as mp
import json
import os
import glob

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()


video_folder = 'C:/Users/user/Desktop/video/broken'
result_folder = 'C:/Users/user/Desktop/video/result'

if not os.path.exists(result_folder):
    os.makedirs(result_folder)

for video_file in glob.glob(os.path.join(video_folder, '*.mp4')):
    cap = cv2.VideoCapture(video_file)

    pose_landmarks_data = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = {}
            for i, landmark in enumerate(results.pose_landmarks.landmark):
                landmarks[f'x{i}'] = landmark.x
                landmarks[f'y{i}'] = landmark.y
                landmarks[f'z{i}'] = landmark.z
                landmarks[f'vis{i}'] = landmark.visibility
            pose_landmarks_data.append(landmarks)

    cap.release()

    video_name = os.path.basename(video_file)
    json_file = os.path.join(result_folder, 'abnormal_' + video_name.replace('.mp4', '.json'))
    with open(json_file, 'w') as f:
        json.dump(pose_landmarks_data, f)

In [26]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()


video_folder = 'C:/Users/user/Desktop/video/buy'
result_folder = 'C:/Users/user/Desktop/video/result'

if not os.path.exists(result_folder):
    os.makedirs(result_folder)

for video_file in glob.glob(os.path.join(video_folder, '*.mp4')):
    cap = cv2.VideoCapture(video_file)

    pose_landmarks_data = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = {}
            for i, landmark in enumerate(results.pose_landmarks.landmark):
                landmarks[f'x{i}'] = landmark.x
                landmarks[f'y{i}'] = landmark.y
                landmarks[f'z{i}'] = landmark.z
                landmarks[f'vis{i}'] = landmark.visibility
            pose_landmarks_data.append(landmarks)

    cap.release()

    video_name = os.path.basename(video_file)
    json_file = os.path.join(result_folder, 'normal_' + video_name.replace('.mp4', '.json'))
    with open(json_file, 'w') as f:
        json.dump(pose_landmarks_data, f)

In [2]:
pip install tensorflow numpy sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [4]:
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

result_folder = 'C:/Users/user/Desktop/video/result'


data = []
labels = []  

for json_file in os.listdir(result_folder):
    if json_file.endswith('.json'):
        with open(os.path.join(result_folder, json_file), 'r') as f:
            landmarks_data = json.load(f)
        
        # 데이터 전처리 로직
        processed_data = np.array([list(frame.values()) for frame in landmarks_data])
        data.append(processed_data)

        # 라벨링
        if 'abnormal' in json_file:
            label = 'abnormal'  # 이상 행동
        else:
            label = 'normal'  # 정상 행동
        labels.append(label)

max_frames = max(len(sequence) for sequence in data)
# 패딩
padded_data = []
for sequence in data:
    sequence_length = len(sequence)
    if sequence_length < max_frames:
        padding = [np.zeros_like(sequence[0]) for _ in range(max_frames - sequence_length)]
        padded_sequence = np.concatenate((sequence, padding), axis=0)
    else:
        padded_sequence = np.array(sequence)  
    padded_data.append(padded_sequence)

max_length = max(len(sequence) for sequence in padded_data)
    
final_data = []
for sequence in padded_data:
    if len(sequence) < max_length:
        padding_length = max_length - len(sequence)
        padding = [np.zeros_like(sequence[0]) for _ in range(padding_length)]
        padded_sequence = np.concatenate((sequence, padding), axis=0)
        final_data.append(padded_sequence)
    else:
        final_data.append(sequence)
    
# 최종 데이터 셋
X = np.array(final_data)
y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = np.array([1 if label == 'abnormal' else 0 for label in y_train])
y_test = np.array([1 if label == 'abnormal' else 0 for label in y_test])

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
5/5 [==============================] - 7s 531ms/step - loss: 0.6631 - accuracy: 0.5486 - val_loss: 0.6767 - val_accuracy: 0.5000
Epoch 2/50
5/5 [==============================] - 1s 269ms/step - loss: 0.6284 - accuracy: 0.5694 - val_loss: 0.6545 - val_accuracy: 0.5000
Epoch 3/50
5/5 [==============================] - 1s 290ms/step - loss: 0.6041 - accuracy: 0.5694 - val_loss: 0.6123 - val_accuracy: 0.5000
Epoch 4/50
5/5 [==============================] - 1s 286ms/step - loss: 0.5135 - accuracy: 0.5694 - val_loss: 0.4713 - val_accuracy: 1.0000
Epoch 5/50
5/5 [==============================] - 1s 291ms/step - loss: 0.4722 - accuracy: 0.8889 - val_loss: 0.5409 - val_accuracy: 0.7500
Epoch 6/50
5/5 [==============================] - 1s 289ms/step - loss: 0.4993 - accuracy: 0.7292 - val_loss: 0.5317 - val_accuracy: 0.7500
Epoch 7/50
5/5 [==============================] - 1s 289ms/step - loss: 0.4866 - accuracy: 0.7292 - val_loss: 0.4724 - val_accuracy: 0.7778
Epoch 8/50
5/5 [====

In [40]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

2/2 [==============================] - 0s 56ms/step - loss: 0.0230 - accuracy: 1.0000
Loss: 0.02301190234720707, Accuracy: 1.0


In [47]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 테스트 해볼 새로운 동영상 파일
new_video_file = 'C:/Users/user/Desktop/video/broken/C_3_8_55_BU_DYB_10-17_14-19-01_CC_RGB_DF2_M3.mp4'
cap = cv2.VideoCapture(new_video_file)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = []
        for landmark in results.pose_landmarks.landmark:
            landmarks.extend([landmark.x, landmark.y, landmark.z])

        # 데이터 길이 조정
        required_length = 601 * 132
        if len(landmarks) < required_length:
            # 부족한 경우 패딩 추가
            landmarks += [0] * (required_length - len(landmarks))
        elif len(landmarks) > required_length:
            # 초과하는 경우 잘라내기
            landmarks = landmarks[:required_length]

        # 모델에 입력할 데이터 준비
        input_data = np.array([landmarks], dtype=np.float32).reshape(-1, 601, 132)
        
        # 모델 예측
        prediction = model.predict(input_data)
        
        if prediction[0][0] > 0.5:
            label = 'Abnormal'
            text_color = (0, 0, 255)  # 빨간색 (B, G, R)
            font_scale = 1.5  # Abnormal일 때 더 큰 글씨 크기
        else:
            label = 'Normal'
            text_color = (255, 0, 0)  # 파란색 (B, G, R)
            font_scale = 1  # Normal일 때 기본 글씨 크기

        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 3)


    cv2.imshow('Pose Detection', frame)
    if cv2.waitKey(1) & 0xFF == 27:  
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 48ms/step


In [ ]:
# 모델 저장
model.save('path/to/save/model.h5')

# 나중에 모델을 다시 로드할 때
from tensorflow.keras.models import load_model
loaded_model = load_model('path/to/save/model.h5')

# 3D CNN
# 오토인코더

In [ ]:
# 3D CNN
# 각 비디오에서 연속된 프레임을 추출
# 이상행동의 프레임을 라벨링 지정
# 비디오 프레임 크기 조정, 정규화 등 전처리 필요

In [5]:
import cv2
import os
# 동영상이 있는 폴더
video_path = 'C:/Users/user/Desktop/video/broken/C_3_8_53_BU_SMC_10-14_13-30-48_CB_RGB_DF2_F3.mp4' 
frames_dir = 'C:/Users/user/Desktop/video/frames'

if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)     # 폴더 자동 생성

cap = cv2.VideoCapture(video_path)
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_path = f'{frames_dir}/frame_{frame_num}.jpg'

    
    if frame is not None:
        cv2.imwrite(frame_path, frame)
        frame_num += 1

cap.release()

In [6]:
import cv2
import os
import numpy as np

def load_frame(filename, width, height):

    frame = cv2.imread(filename)
    frame = cv2.resize(frame, (128, 128))  #  (width, height) = (128, 128)
    return frame

frame_folder = 'C:/Users/user/Desktop/video/frames'
frame_width = 128  # 조정할 프레임의 너비
frame_height = 128  # 조정할 프레임의 높이
sequence_length = 10  # 시퀀스 길이
sequences = []
labels = []

for start_frame in range(0, 180, sequence_length):
    end_frame = start_frame + sequence_length
    sequence = []

    for frame_number in range(start_frame, end_frame):
        frame_path = os.path.join(frame_folder, f'frame_{frame_number}.jpg')
        frame = load_frame(frame_path, frame_width, frame_height)
        sequence.append(frame)

    if 74 <= start_frame <= 147 or 74 <= end_frame <= 147:
        labels.append('abnormal')
    else:
        labels.append('normal')

    sequences.append(sequence)

In [7]:
X = np.array(sequences)  # (시퀀스 수, 시퀀스 길이, 높이, 너비, 채널 수)
y = np.array([1 if label == 'abnormal' else 0 for label in labels])  # 이진 레이블

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(10, 128, 128, 3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 19.5357 - accuracy: 0.5455 - val_loss: 940.2484 - val_accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 1753.4644 - accuracy: 0.4545 - val_loss: 223.6887 - val_accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 428.6719 - accuracy: 0.4545 - val_loss: 140.4767 - val_accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 121.4214 - accuracy: 0.5455 - val_loss: 61.0738 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 30.7511 - accuracy: 0.4545 - val_loss: 4.0700 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 2.0374 - accuracy: 0.7273 - val_loss: 10.9537 - val_accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 14.7566 - accuracy: 0.4545 - val_loss: 2.0892 - val_accuracy: 0.6667
Epoch 8/10
1/1 [=======

In [11]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

1/1 [==============================] - 0s 104ms/step - loss: 0.7028 - accuracy: 0.7500
Test accuracy: 0.75


In [12]:
model.save('3D_CNN_model.h5')

C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
import cv2
import numpy as np
from keras.models import load_model

# 프레임 전처리 함수 정의
def preprocess_frame(frame, target_size=(128, 128)):
    frame = cv2.resize(frame, target_size)  # 프레임 리사이징
    frame = frame / 255.0  # 정규화 (필요한 경우)
    return frame

# 학습된 모델 로드
model = load_model('3D_CNN_model.h5')

# 동영상 파일 로드
video_file = 'C:/Users/user/Desktop/video/broken/C_3_8_55_BU_DYB_10-17_14-19-01_CC_RGB_DF2_M3.mp4'
cap = cv2.VideoCapture(video_file)

sequence = []  # 현재 프레임 시퀀스
sequence_length = 10  # 모델에 입력할 시퀀스 길이

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 전처리
    processed_frame = preprocess_frame(frame)

    # 시퀀스 업데이트
    sequence.append(processed_frame)
    if len(sequence) < sequence_length:
        continue
    
    # 모델 입력을 위한 시퀀스 준비
    input_frames = np.expand_dims(np.array(sequence), axis=0)

    # 모델 예측
    prediction = model.predict(input_frames)
    if prediction[0][0] > 0.5:
        text = 'Abnormal'
        color = (0, 0, 255)
    else:
        text = 'Normal'
        color = (255, 0, 0)

    # 텍스트 표시
    cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # 프레임 표시
    cv2.imshow('Frame', frame)

    # 시퀀스에서 가장 오래된 프레임 제거
    sequence.pop(0)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 38ms/step
